In [ ]:
#This is for the Movie Mayhem Project!

In [ ]:
#This is for loading the modules we want to use for the project.
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats as stats 

In [ ]:
#This is for loading the CSV and parsing into a DF
movie_dataset = pd.read_csv("MoviesOnStreamingPlatforms_updated.csv")
movie_dataset

In [ ]:
movie_dataset = movie_dataset.set_index('ID')
movie_dataset = movie_dataset.drop(['Unnamed: 0', 'Type'], axis=1)

#Checked to see how many rows would be lost with dropping movies withour Rotten Tomatoe scores, 
#we saw it would reduce our dataset by about 11,000 rows and decided to utilize just IMDb

movie_dataset = movie_dataset.drop(['Rotten Tomatoes'], axis=1)
movie_dataset

In [ ]:
# This block is for the team's questions

<b>Question:</b> Do movies aimed at 18+ age have a higher IMDB rating?

In [ ]:
#Cleaning the data for the age analysis
movie_dataset_age_data = movie_dataset.copy()
movie_dataset_age_data_clean = movie_dataset_age_data.dropna(subset=['Age', 'IMDb'])
movie_dataset_age_data_clean

In [ ]:
# Creating column to compare over 18 vs under 18 movies
movie_dataset_age_data_clean['Age Range'] = np.where(movie_dataset_age_data_clean['Age'] == '18+', '18+', 'Under 18')

# Didn't end up needed group by had to parse out data seperately
movie_dataset_age_data_clean_groupby = movie_dataset_age_data_clean.groupby('Age Range')[['IMDb']].mean()
movie_dataset_age_data_clean_groupby


over_18 = movie_dataset_age_data_clean.loc[movie_dataset_age_data_clean['Age Range'] == '18+', 'IMDb']
under_18 = movie_dataset_age_data_clean.loc[movie_dataset_age_data_clean['Age Range'] != '18+', 'IMDb']


In [ ]:
# Checking for outliers in Age Range data

age_range_data = [over_18, under_18]

fig1, ax1 = plt.subplots()
ax1.set_xticklabels(['Over 18', 'Under 18'])
ax1.set_ylabel("IMDb Rating")
ax1.set_title("Movie Ratings by Age Range")
ax1.boxplot(age_range_data, 0, '*')
plt.show()

print("Based on the boxplot we've uncovered many outliers among both age ranges.") 
print('We will run a two Hypothesis test. 1 including outliers and 1 not including outliers')



In [ ]:
# First Age Range Hypothesis Test

# Null Hypothesis: There is no difference in IMDb ratings based on age

print('Our hypothesis is that under 18 will have higher ratings then over 18')

print('We construct a null hypothesis that states there is no difference in ratings based on age')

print("We test this null hypothesis with a Student's t-test")


print('p-value: ' +str(stats.ttest_ind(over_18, under_18, equal_var=False)[1]))

print('----------------------------------')

print('Based on the p-value, we see that the difference in ratings is statistically significant.')

print('We now believe under 18 IMDb ratings are higher than Over 18')

# Increases our confidence that under 18 has higher ratings then over 18
                

In [ ]:
# Second Age Range Hypothesis Test, removing outliers

over_18_quartiles = over_18.quantile([.25,.5,.75])
over_18_lowerq = over_18_quartiles[0.25]
over_18_upperq = over_18_quartiles[0.75]
over_18_iqr = over_18_upperq-over_18_lowerq

over_18_lower_bound = over_18_lowerq - (1.5*over_18_iqr)
over_18_upper_bound = over_18_upperq + (1.5*over_18_iqr)

over_18_df = over_18.to_frame()


filtered_over18_df = over_18_df.loc[(over_18_df['IMDb'] >= over_18_lower_bound) & (over_18_df['IMDb'] <= over_18_upper_bound)]

filtered_over18_df.head()

In [ ]:
# Second Age Range Hypothesis Test, removing outliers
under_18_quartiles = under_18.quantile([.25,.5,.75])
under_18_lowerq = under_18_quartiles[0.25]
under_18_upperq = under_18_quartiles[0.75]
under_18_iqr = under_18_upperq-under_18_lowerq

under_18_lower_bound = under_18_lowerq - (1.5*under_18_iqr)
under_18_upper_bound = under_18_upperq + (1.5*under_18_iqr)

#print(over_18_lower_bound)
#print(over_18_upper_bound)

#print(type(over_18))

under_18_df = under_18.to_frame()


filtered_under18_df = under_18_df.loc[(under_18_df['IMDb'] >= under_18_lower_bound) & (under_18_df['IMDb'] <= under_18_upper_bound)]

filtered_under18_df.head()

In [ ]:
# Second Age Range Hypothesis Test

# Null Hypothesis: There is no difference in IMDb ratings based on age

print('Our hypothesis is that under 18 will have higher ratings then over 18')

print('We construct a null hypothesis that states there is no difference in ratings based on age')

print("We test this null hypothesis with a Student's t-test")


print('p-value: ' +str(stats.ttest_ind(filtered_over18_df, filtered_under18_df, equal_var=False)[1]))

print('----------------------------------')

print('Based on the p-value, we see that the difference in ratings is statistically significant.')

print('We now believe under 18 IMDb ratings are higher than Over 18')

# Increases our confidence that under 18 has higher ratings then over 18
                

In [ ]:
#This block is for Robertos Code

In [ ]:
# My own copy of the original Data
robertos_dataset = movie_dataset.copy()
robertos_dataset

In [ ]:
#robertos_dataset.columns
# Selecting only ID, IMDb, Genres columns
robertos_dataset_simplified = robertos_dataset[["Title","IMDb","Genres"]]
# robertos_dataset.str.split(',', expand=True)
# robertos_dataset
#robertos_dataset_simplified
print(f"robertos_dataset_simplified={robertos_dataset_simplified.size}")
robertos_dataset_cleaned = robertos_dataset_simplified.dropna(how='any')
print(f"robertos_dataset_cleaned={robertos_dataset_cleaned.size}")

In [ ]:
#Creating a new dataframe containing every movie with one genre per row
robertos_dataset_one_genre_by_movie = pd.DataFrame(columns = ['Title', 'IMDb', 'Genre']) 
# For each row in original df split Genres column by ',' and create a new row per genre per movie
for index, row in robertos_dataset_cleaned.iterrows():
    list_of_genres = row["Genres"].split(",")
    for genre in list_of_genres:
        new_row = {'Title':row.Title, 'IMDb': row.IMDb, 'Genre': genre.strip()}
        robertos_dataset_one_genre_by_movie = robertos_dataset_one_genre_by_movie.append(new_row, ignore_index=True)
        # print(f"genre --{genre}--{robertos_dataset_one_genre_by_movie.index}")

robertos_dataset_one_genre_by_movie['Genre'].replace('',np.nan, inplace=True)

robertos_dataset_one_genre_by_movie.dropna(how='any', inplace=True)
robertos_dataset_one_genre_by_movie

In [ ]:
robertos_dataset_one_genre_by_movie

In [ ]:
robertos_dataset_grouped = robertos_dataset_one_genre_by_movie.groupby(by=["Genre"]).mean()
robertos_dataset_grouped.sort_values(by=["IMDb"], ascending=False)

In [ ]:
# Checking for outliers in Age Range data
news = robertos_dataset_one_genre_by_movie.loc[robertos_dataset_one_genre_by_movie['Genre'] == 'News', 'IMDb']
documentary = robertos_dataset_one_genre_by_movie.loc[robertos_dataset_one_genre_by_movie['Genre'] == 'Documentary', 'IMDb']
biography = robertos_dataset_one_genre_by_movie.loc[robertos_dataset_one_genre_by_movie['Genre'] == 'Biography', 'IMDb']
history = robertos_dataset_one_genre_by_movie.loc[robertos_dataset_one_genre_by_movie['Genre'] == 'History', 'IMDb']
music = robertos_dataset_one_genre_by_movie.loc[robertos_dataset_one_genre_by_movie['Genre'] == 'Music', 'IMDb']
short = robertos_dataset_one_genre_by_movie.loc[robertos_dataset_one_genre_by_movie['Genre'] == 'Short', 'IMDb']
sport = robertos_dataset_one_genre_by_movie.loc[robertos_dataset_one_genre_by_movie['Genre'] == 'Sport', 'IMDb']
film_Noir = robertos_dataset_one_genre_by_movie.loc[robertos_dataset_one_genre_by_movie['Genre'] == 'Film-Noir', 'IMDb']
animation = robertos_dataset_one_genre_by_movie.loc[robertos_dataset_one_genre_by_movie['Genre'] == 'Animation', 'IMDb']
war = robertos_dataset_one_genre_by_movie.loc[robertos_dataset_one_genre_by_movie['Genre'] == 'War', 'IMDb']
musical = robertos_dataset_one_genre_by_movie.loc[robertos_dataset_one_genre_by_movie['Genre'] == 'Musical', 'IMDb']
game_Show = robertos_dataset_one_genre_by_movie.loc[robertos_dataset_one_genre_by_movie['Genre'] == 'Game-Show', 'IMDb']
drama = robertos_dataset_one_genre_by_movie.loc[robertos_dataset_one_genre_by_movie['Genre'] == 'Drama', 'IMDb']
romance = robertos_dataset_one_genre_by_movie.loc[robertos_dataset_one_genre_by_movie['Genre'] == 'Romance', 'IMDb']
family = robertos_dataset_one_genre_by_movie.loc[robertos_dataset_one_genre_by_movie['Genre'] == 'Family', 'IMDb']

comedy = robertos_dataset_one_genre_by_movie.loc[robertos_dataset_one_genre_by_movie['Genre'] == 'Comedy', 'IMDb']
western = robertos_dataset_one_genre_by_movie.loc[robertos_dataset_one_genre_by_movie['Genre'] == 'Western', 'IMDb']
crime = robertos_dataset_one_genre_by_movie.loc[robertos_dataset_one_genre_by_movie['Genre'] == 'Crime', 'IMDb']
talk_Show = robertos_dataset_one_genre_by_movie.loc[robertos_dataset_one_genre_by_movie['Genre'] == 'Talk-Show', 'IMDb']
fantasy = robertos_dataset_one_genre_by_movie.loc[robertos_dataset_one_genre_by_movie['Genre'] == 'Fantasy', 'IMDb']
mystery = robertos_dataset_one_genre_by_movie.loc[robertos_dataset_one_genre_by_movie['Genre'] == 'Mystery', 'IMDb']
adventure = robertos_dataset_one_genre_by_movie.loc[robertos_dataset_one_genre_by_movie['Genre'] == 'Adventure', 'IMDb']
# reality_TV = robertos_dataset_one_genre_by_movie.loc[robertos_dataset_one_genre_by_movie['Reality-TV'] == 'Short', 'IMDb']


In [ ]:
genre_range_data = [news, documentary]

fig1, ax1 = plt.subplots()
ax1.set_xticklabels(['News', 'Documentary'])
ax1.set_ylabel("IMDb Rating")
ax1.set_title("Movie Ratings by Genre")
ax1.boxplot(genre_range_data, 0, '*')
plt.show()

print("Based on the boxplot we've uncovered many outliers among both age ranges.") 
print('We will run a two Hypothesis test. 1 including outliers and 1 not including outliers')

In [ ]:
#This block is for Tanners code

<b>Question:</b> Do movies on multiple streaming platforms have higher IMDb ratings?


In [ ]:
platforms_df = movie_dataset.copy()

platforms_df['No. Streaming Platforms'] = platforms_df['Netflix']+platforms_df['Hulu']+platforms_df['Disney+']+platforms_df['Prime Video']
platforms_df['Multiple Steaming Platforms'] = np.where(platforms_df['No. Streaming Platforms'] == 1, 'One Platform', 'Multiple Platforms')

platforms_groupby = platforms_df.groupby('Multiple Steaming Platforms')[['IMDb']].mean()


platforms_groupby.plot(kind='bar', rot=.1)
plt.title('Do Movies on Multiple Streaming Platforms Having Higher IMDb Ratings?')
plt.ylabel('IMDb')
plt.xlabel('# of Streaming Platforms')
plt.ylim(0,10)

plt.show()

print(platforms_groupby)


#### NOTES ####

# Based on this graph it's looking like there's not a huge difference in rating if movies 
# have been on multiple platforms vs just one

# Could be interesting to add a couple graphs by Popular Genres to see if that makes any difference



In [ ]:
#This block is for Davids code
# Which streaming platform has the highest average ratings?
# dropping NAN's from IMDb column to allow data to be worked.
stream_dataset = movie_dataset.copy()
stream_dataset_clean = stream_dataset.dropna(subset=['IMDb'])
disney_df = stream_dataset_clean.loc[stream_dataset_clean["Disney+"] == 1, :]
disney_clean = stream_dataset_clean.loc[stream_dataset_clean['Disney+'] == 1, 'IMDb']
disney_avg = disney_df["IMDb"].mean()
netflix_df = stream_dataset_clean.loc[stream_dataset_clean["Netflix"] == 1, :]
netflix_clean = stream_dataset_clean.loc[stream_dataset_clean['Netflix'] == 1, 'IMDb']
netflix_avg = netflix_df["IMDb"].mean()
hulu_df = stream_dataset_clean.loc[stream_dataset_clean["Hulu"] == 1, :]
hulu_clean = stream_dataset_clean.loc[stream_dataset_clean['Hulu'] == 1, 'IMDb']
hulu_avg = hulu_df["IMDb"].mean()
prime_df = stream_dataset_clean.loc[stream_dataset_clean["Prime Video"] == 1, :]
prime_clean = stream_dataset_clean.loc[stream_dataset_clean['Prime Video'] == 1, 'IMDb']
prime_avg = prime_df["IMDb"].mean()

Services = ["Disney+", "Netflix", "Hulu", "Prime Video"]
services_avg = [disney_avg, netflix_avg, hulu_avg, prime_avg]
x_axis = np.arange(len(services_avg))

In [ ]:
plt.bar(x_axis, services_avg, color="b", align="center")
plt.xlim(-0.75, len(x_axis)-0.25)
plt.ylim(4, 8)
plt.title("Streaming Services Average IMDb Ratings")
plt.xlabel("Streaming Service")
plt.ylabel("IMDb Rating")
tick_locations = [value for value in x_axis]
plt.xticks(tick_locations, Services)
plt.locator_params(axis='y', nbins=10)
plt.grid(linestyle='--')

In [ ]:
services_data = [disney_clean, netflix_clean, hulu_clean, prime_clean]

fig1, ax1 = plt.subplots()
ax1.set_xticklabels(['Disney+', 'Netflix', 'Hulu', 'Prime Video'])
ax1.set_ylabel("IMDb Rating")
ax1.set_title("Movie Ratings by Streaming Service")
ax1.boxplot(services_data, 0, '*')
plt.show()
# Showing that Disney+ had the highest rating, but if outliers where removed then a different service might have a higer Mean IMDb score.

In [ ]:
#This block is for Matts code

In [ ]:
#The hypothesis for this test is that movie ratings will have a recency bias based on decade. 
#That decade a movie is made in has no bearing on the ratings of that movie


#After reviewing the data, we decided we needed to ensure that a decade had at least 300 movies in it
#this exluded decades before 1930, we also wanted to exclude 2020 and 2021 since that isnt a complete decade.


#First we copied our dataframe, then we dropped years before 1930 and after 2019
decades_df = movie_dataset.copy()
decades_clean_df = decades_df.dropna(subset=['IMDb'])
decades_clean_df = decades_clean_df[decades_clean_df['Year'] > 1930]
decades_clean_df = decades_clean_df[decades_clean_df['Year'] < 2020]

#Then we assigned the ranges for each decade
conditions = [
    (decades_clean_df['Year'] >= 1930) & (decades_clean_df['Year'] < 1940),
    (decades_clean_df['Year'] >= 1940) & (decades_clean_df['Year'] < 1950),
    (decades_clean_df['Year'] >= 1950) & (decades_clean_df['Year'] < 1960),
    (decades_clean_df['Year'] >= 1960) & (decades_clean_df['Year'] < 1970),
    (decades_clean_df['Year'] >= 1970) & (decades_clean_df['Year'] < 1980),
    (decades_clean_df['Year'] >= 1980) & (decades_clean_df['Year'] < 1990),
    (decades_clean_df['Year'] >= 1990) & (decades_clean_df['Year'] < 2000),
    (decades_clean_df['Year'] >= 2000) & (decades_clean_df['Year'] < 2010),
    (decades_clean_df['Year'] >= 2010) & (decades_clean_df['Year'] < 2020)
]

#And then created the labels for those decades.
decades = ["1930's","1940's","1950's","1960's","1970's","1980's","1990's","2000's","2010's"]

#Then we added a decades column to our dataframe
decades_clean_df['Decade'] = np.select(conditions, decades)
#decades_clean_df = decades_clean_df[decades_clean_df['Decade'] != 0]

#And made a dataframe grouped by the decades
decades_grouped_df = decades_clean_df.groupby('Decade')[('IMDb')].mean()

#To better plot the data i turned each decade into its own list.
IMDb_data_1930 = decades_clean_df[decades_clean_df['Decade'] == "1930's"]
IMDb_data_1940 = decades_clean_df[decades_clean_df['Decade'] == "1940's"]
IMDb_data_1950 = decades_clean_df[decades_clean_df['Decade'] == "1950's"]
IMDb_data_1960 = decades_clean_df[decades_clean_df['Decade'] == "1960's"]
IMDb_data_1970 = decades_clean_df[decades_clean_df['Decade'] == "1970's"]
IMDb_data_1980 = decades_clean_df[decades_clean_df['Decade'] == "1980's"]
IMDb_data_1990 = decades_clean_df[decades_clean_df['Decade'] == "1990's"]
IMDb_data_2000 = decades_clean_df[decades_clean_df['Decade'] == "2000's"]
IMDb_data_2010 = decades_clean_df[decades_clean_df['Decade'] == "2010's"]

IMDb_data_1930 = IMDb_data_1930['IMDb'].tolist()
IMDb_data_1940 = IMDb_data_1940['IMDb'].tolist()
IMDb_data_1950 = IMDb_data_1950['IMDb'].tolist()
IMDb_data_1960 = IMDb_data_1960['IMDb'].tolist()
IMDb_data_1970 = IMDb_data_1970['IMDb'].tolist()
IMDb_data_1980 = IMDb_data_1980['IMDb'].tolist()
IMDb_data_1990 = IMDb_data_1990['IMDb'].tolist()
IMDb_data_2000 = IMDb_data_2000['IMDb'].tolist()
IMDb_data_2010 = IMDb_data_2010['IMDb'].tolist()

decade_data = [IMDb_data_1930, IMDb_data_1940, IMDb_data_1950, IMDb_data_1960, IMDb_data_1970,
               IMDb_data_1980, IMDb_data_1990, IMDb_data_2000, IMDb_data_2010]

#Then we plotted a box plot to get a better visualization of our data.
fig1, ax1 = plt.subplots()
ax1.set_xticklabels(decades)
ax1.set_ylabel("IMDb Rating")
ax1.set_title("Movie Ratings by Decade")
ax1.boxplot(decade_data, 0, '*')
plt.show()

#Next we ran a anova test to try to disprove our null hypothesis
stats.f_oneway(IMDb_data_1930, IMDb_data_1940, IMDb_data_1950, IMDb_data_1960, IMDb_data_1970,
               IMDb_data_1980, IMDb_data_1990, IMDb_data_2000, IMDb_data_2010) 

print('p-value: ' +str(stats.f_oneway(IMDb_data_1930, IMDb_data_1940, IMDb_data_1950, IMDb_data_1960, IMDb_data_1970,
               IMDb_data_1980, IMDb_data_1990, IMDb_data_2000, IMDb_data_2010)))

#Based on the anova test we ran, we can disprove the null hypothesis that the decade a movie was released in has no bearing on its review score.

